In [167]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
from tqdm import tqdm
%matplotlib inline
pd.set_option('display.max_colwidth', 300)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

In [168]:
textdata = []
with open('/content/drive/MyDrive/booksummaries.txt', 'r') as f:
  reader = csv.reader(f, dialect='excel-tab') 
  for row in tqdm(reader):
    textdata.append(row)
      

16559it [00:00, 26150.94it/s]


In [169]:
book_id=[]
genre = []
name=[]
summary=[]
author=[]

for x in tqdm(textdata):
  book_id.append(x[0])
  genre.append(x[5])
  name.append(x[2])
  summary.append(x[6])
  author.append(x[3])

book_info=pd.DataFrame({'Book_id':book_id, 'Name':name, 'Author':author, 'Genre':genre, 'Summary':summary})
book_info.head()

100%|██████████| 16559/16559 [00:00<00:00, 699783.17it/s]


,Book_id,Name,Author,Genre,Summary
0,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p..."
1,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and..."
2,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fiction"", ""/m/0pym5"": ""Absurdist fiction"", ""/m/05hgj"": ""Novel""}","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t..."
3,1756,An Enquiry Concerning Human Understanding,David Hume,,"The argument of the Enquiry proceeds by a series of incremental steps, separated into chapters which logically succeed one another. After expounding his epistemology, Hume explains how to apply his principles to specific topics. In the first section of the Enquiry, Hume provides a rough introdu..."
4,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""Speculative fiction"", ""/m/01hmnh"": ""Fantasy"", ""/m/02xlf"": ""Fiction""}","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti..."


In [170]:
book_info.drop(book_info[book_info['Genre']==''].index, inplace=True)
book_info[book_info['Genre']=='']

,Book_id,Name,Author,Genre,Summary


In [171]:
json.loads(book_info['Genre'][0]).values()

dict_values(['Roman à clef', 'Satire', "Children's literature", 'Speculative fiction', 'Fiction'])

In [172]:
Genre_data=[]
for x in book_info['Genre']:
  Genre_data.append(list(json.loads(x).values()))
book_info['Genre_process']=Genre_data

In [173]:
all = sum(Genre_data,[])
len(set(all))

227

In [174]:
def process_summary(text):
  text = re.sub("\'","", text)
  text = re.sub("[^a-zA-Z]"," ", text)
  text = ' '.join(text.split())
  text = text.lower()
  return text

In [175]:
book_info['process_summary'] = book_info['Summary'].apply(lambda x: process_summary(x))
book_info.head()

,Book_id,Name,Author,Genre,Summary,Genre_process,process_summary
0,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p...","[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]",old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy t...
1,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and...","[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]",alex a teenager living in near future england leads his gang on nightly orgies of opportunistic random ultra violence alexs friends droogs in the novels anglo russian slang nadsat are dim a slow witted bruiser who is the gangs muscle georgie an ambitious second in command and pete who mostly pla...
2,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fiction"", ""/m/0pym5"": ""Absurdist fiction"", ""/m/05hgj"": ""Novel""}","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t...","[Existentialism, Fiction, Absurdist fiction, Novel]",the text of the plague is divided into five parts in the town of oran thousands of rats initially unnoticed by the populace begin to die in the streets a hysteria develops soon afterward causing the local newspapers to report the incident authorities responding to public pressure order the colle...
4,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""Speculative fiction"", ""/m/01hmnh"": ""Fantasy"", ""/m/02xlf"": ""Fiction""}","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti...","[Hard science fiction, Science Fiction, Speculative fiction, Fantasy, Fiction]",the novel posits that space around the milky way is divided into concentric layers called zones each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement the innermost the unthinking depths surrounds the galactic core ...
5,2152,All Quiet on the Western Front,Erich Maria Remarque,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman \u00e0 clef""}","The book tells the story of Paul Bäumer, a German soldier who—urged on by his school teacher—joins the German army shortly after the start of World War I. Bäumer arrives at the Western Front with his friends and schoolmates (Tjaden, Müller, Kropp and a numbe

In [176]:
import nltk
nltk.download("stopwords")
stopwords_set = set(stopwords.words('english'))
def remove(text):
    stopword_process = [w for w in text.split() if not w in stopwords_set]
    return ' '.join(stopword_process)

book_info['process_summary'] = book_info['process_summary'].apply(lambda x: remove(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [177]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(book_info['Genre_process'])
y = multilabel_binarizer.transform(book_info['Genre_process'])

In [178]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(book_info['process_summary'],y, test_size=0.2)

In [179]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [180]:
X_train = tfidf_vectorizer.fit_transform(x_train)

In [181]:
X_val = tfidf_vectorizer.transform(x_val)

In [182]:
from sklearn.linear_model import LogisticRegression
LogR = LogisticRegression()

In [183]:
from sklearn.multiclass import OneVsRestClassifier
OVR = OneVsRestClassifier(LogR)

In [184]:
OVR.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 24 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 30 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 41 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 67 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 79 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 131 is present in all training 

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [185]:
from sklearn.metrics import f1_score, accuracy_score
y_pred = OVR.predict(X_val)
f1_score(y_val, y_pred, average='micro')

0.3677792831629511

In [186]:
accuracy_score(y_val, y_pred)

0.07901907356948229

In [187]:
pred_prob = OVR.predict_proba(X_val)

In [188]:
t=0.3
predp = (pred_prob >= t).astype(int)
f1_score(y_val, predp, average='micro')

0.5303985089198544

In [189]:
accuracy_score(y_val, predp)

0.11093810821331257

In [190]:
def predict(m):
  m = process_summary(m)
  m = remove(m)
  m_vec = tfidf_vectorizer.transform([m])
  m_pred = OVR.predict(m_vec)
  return multilabel_binarizer.inverse_transform(m_pred)

In [191]:
for x in range(10):
  a = x_val.sample(1).index[0]
  print("Book: ", book_info['Name'][a], "\nPredicted Genre: ", predict(x_val[a]))
  print("Excpected Genre: ", book_info['Genre_process'][a], "\n")

Book:  If I Stay 
Predicted Genre:  [()]
Excpected Genre:  ['Young adult literature', 'Fiction'] 

Book:  The Land of Painted Caves 
Predicted Genre:  [('Fiction', 'Speculative fiction')]
Excpected Genre:  ['Alternate history', 'Speculative fiction', 'Historical novel'] 

Book:  The Hippopotamus Pool 
Predicted Genre:  [()]
Excpected Genre:  ['Mystery', 'Historical novel'] 

Book:  Prophets of the Dark Side 
Predicted Genre:  [('Science Fiction', 'Speculative fiction')]
Excpected Genre:  ['Science Fiction'] 

Book:  Obsidian Butterfly 
Predicted Genre:  [('Mystery', 'Speculative fiction')]
Excpected Genre:  ['Science Fiction', 'Horror', 'Speculative fiction', 'Mystery', 'Fantasy', 'Fiction'] 

Book:  Without Remorse 
Predicted Genre:  [('Fiction',)]
Excpected Genre:  ['Thriller', 'Mystery', 'Novel', 'Fiction', 'Suspense'] 

Book:  Sir Nobonk and the Terrible Dreadful Awful Naughty Nasty Dragon 
Predicted Genre:  [('Fantasy', 'Speculative fiction')]
Excpected Genre:  ["Children's litera

In [192]:
# References:

# https://www.ics.uci.edu/~smyth/courses/cs175/text_data_sets.html
# http://www.cs.cmu.edu/~dbamman/booksummaries.html
# https://www.geeksforgeeks.org/detect-an-unknown-language-using-python/
# https://towardsdatascience.com/an-illustrated-guide-to-the-poisson-regression-model-50cccba15958
# https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html
# https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
# https://www.analyticsvidhya.com/blog/2019/04/predicting-movie-genres-nlp-multi-label-classification/
# https://stackoverflow.com/questions/26693736/nltk-and-stopwords-fail-lookuperror
# https://www.kaggle.com/ymaricar/cmu-book-summary-dataset